In [1]:
from src.utils import *

In [2]:
puzzle_input = parse_puzzle_input(4)

In [3]:
puzzle_input[:3]

['17,2,33,86,38,41,4,34,91,61,11,81,3,59,29,71,26,44,54,89,46,9,85,62,23,76,45,24,78,14,58,48,57,40,21,49,7,99,8,56,50,19,53,55,10,94,75,68,6,83,84,88,52,80,73,74,79,36,70,28,37,0,42,98,96,92,27,90,47,20,5,77,69,93,31,30,95,25,63,65,51,72,60,16,12,64,18,13,1,35,15,66,67,43,22,87,97,32,39,82',
 '',
 '10 27 53 91 86']

In [4]:
sample_input = [
'7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1',
'',
'22 13 17 11  0',
' 8  2 23  4 24',
'21  9 14 16  7',
' 6 10  3 18  5',
' 1 12 20 15 19',
'',
' 3 15  0  2 22',
' 9 18 13 17  5',
'19  8  7 25 23',
'20 11 10 24  4',
'14 21 16 12  6',
'',
'14 21 17 24  4',
'10 16 15  9 19',
'18  8 23 26 20',
'22 11 13  6  5',
' 2  0 12  3  7'
]

In [5]:
def parse_drawn_order(line_str):
    return [int(x) for x in line_str.split(',')]

In [31]:
def parse_grid(bingo_board_list):
    rows = [[int(x.strip()) for x in bingo_row.split()] for bingo_row in bingo_board_list]
    col_dict = {k: [] for k in range(len(rows[0]))}
    for row in rows:
        for index, number in enumerate(row):
            col_dict[index].append(number)
    row_set = {frozenset(row) for row in rows}
    col_set = {frozenset(col) for col in col_dict.values()}
    return row_set.union(col_set)


In [33]:
def update_bingo_dict(rolling_list, board_index, bingo_dict):
    bingo_dict[board_index] = parse_grid(rolling_list)
    rolling_list = []
    board_index += 1
    return rolling_list, board_index, bingo_dict

In [62]:
def parse_bingo_input(puzzle_input):
    bingo_dict = {'numbers': parse_drawn_order(puzzle_input[0])}

    rolling_list = []
    board_index = 0

    for input_row in puzzle_input[2:]:
        if input_row == '':
            rolling_list, board_index, bingo_dict = update_bingo_dict(rolling_list, board_index, bingo_dict)
        else:
            rolling_list.append(input_row)
    rolling_list, board_index, bingo_dict = update_bingo_dict(rolling_list, board_index, bingo_dict)

    bingo_dict['n_boards'] = board_index + 1

    return bingo_dict
        


In [48]:
def check_if_valid_bingo_line(bingo_line, drawn_numbers):
    return set(drawn_numbers).issuperset(bingo_line)

In [49]:
def check_if_valid_bingo_board(bingo_board, drawn_numbers):
    return bool(
        sum(
            check_if_valid_bingo_line(bingo_line, drawn_numbers)
            for bingo_line in bingo_board
        )
    )

In [91]:
def find_winning_board(bingo_dict, drawn_numbers):
    bingo = False
    for board_index in range(bingo_dict['n_boards'] - 1):
        if board_index in bingo_dict:   
            if not bingo and check_if_valid_bingo_board(
                bingo_dict[board_index], drawn_numbers
            ):
                bingo = True
                bingo_board_id = board_index
    if bingo:
        return bingo_board_id
    return None


In [69]:
def get_all_board_numbers(bingo_board):
    all_numbers = set()
    for line in bingo_board:
        for number in line:
            all_numbers.add(number)
    return all_numbers

In [77]:
def find_winning_board_score(winning_board, drawn_numbers):
    all_numbers = get_all_board_numbers(winning_board)
    uncalled_numbers = all_numbers.difference(drawn_numbers)
    return sum(uncalled_numbers) * drawn_numbers[-1]

In [121]:
def run_bingo_subsystem(puzzle_input):

    bingo_dict = parse_bingo_input(puzzle_input)

    bingo = False
    drawn_number_index = 4

    while not bingo:

        drawn_number_index += 1
        drawn_numbers = bingo_dict['numbers'][:drawn_number_index]

        winning_board_id = find_winning_board(bingo_dict, drawn_numbers)

        if winning_board_id is not None:
            bingo = True

    return find_winning_board_score(bingo_dict[winning_board_id], drawn_numbers)




In [81]:
run_bingo_subsystem(sample_input)

4512

In [82]:
run_bingo_subsystem(puzzle_input)

38594

## Part 2

In [141]:
def find_board_win_order(bingo_dict_input):

    bingo_dict = bingo_dict_input.copy()

    board_win_order = []

    for drawn_number_index in range(5, len(bingo_dict['numbers'])):
        drawn_numbers = bingo_dict['numbers'][:drawn_number_index]

        winning_board_id = find_winning_board(bingo_dict, drawn_numbers)

        while winning_board_id is not None:
            board_win_order.append(winning_board_id)
            bingo_dict.pop(winning_board_id)
            winning_board_id = find_winning_board(bingo_dict, drawn_numbers)

    return board_win_order

In [131]:
def find_score_given_board(board_id, bingo_dict):

    bingo = False
    drawn_number_index = 4

    while not bingo:

        drawn_number_index += 1
        drawn_numbers = bingo_dict['numbers'][:drawn_number_index]

        if check_if_valid_bingo_board(bingo_dict[board_id], drawn_numbers):

            bingo = True

    return find_winning_board_score(bingo_dict[board_id], drawn_numbers)



In [132]:
def run_sureloss_bingo_subsystem(puzzle_input):

    bingo_dict = parse_bingo_input(puzzle_input)

    worst_board = find_board_win_order(bingo_dict)[-1]

    return find_score_given_board(worst_board, bingo_dict)



In [145]:
run_sureloss_bingo_subsystem(sample_input)

1924

In [146]:
run_sureloss_bingo_subsystem(puzzle_input)

21184